This notebook provides examples to go along with the [textbook](https://underactuated.csail.mit.edu/acrobot.html).  I recommend having both windows open, side-by-side!


In [6]:
import numpy as np
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    LeafSystem,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    ModelVisualizer,
    Parser,
    Simulator,
    StartMeshcat,
)

from underactuated import ConfigureParser, running_as_notebook
from underactuated.meshcat_utils import MeshcatSliders

In [7]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


In [8]:
# just visualize the urdf
visualizer = ModelVisualizer(meshcat=meshcat)
parser=visualizer.parser()
parser.package_map().Add('my_urdf','/home/ram/personal/underactuated_robotics_mit_course/programming_examples/03-Acrobots-Cart-Poles-and-Quadrotors/my_urdf')
urdf_url = "package://my_urdf/sqwip.urdf"
parser.AddModelsFromUrl(urdf_url)
visualizer.Run(loop_once=not running_as_notebook)
meshcat.Delete()
meshcat.DeleteAddedControls()

INFO:drake:Click 'Stop Running' or press Esc to quit


In [9]:
# [x,z,theta,pole_theta,q1,q2,q3,q4,x_dot,z_dot,theta_dot,pole_theta_dot,q1_dot,q2_dot,q3_dot,q4_dot]
base_pose=[0.0,0.3598,0.0] # x z and y theta
base_vel_dot=[0,0,0]   #xdot z dot ytheta dot
pole_theta=[np.pi]
pole_theta_dot=[0]

leg_joint_pos=[np.radians(30),-np.radians(30),np.radians(30),-np.radians(30)]
leg_joint_vel=[0,0,0,0]




x_star_system=base_pose+pole_theta+leg_joint_pos+base_vel_dot+pole_theta+leg_joint_vel

print(x_star_system)

[0.0, 0.3598, 0.0, 3.141592653589793, 0.5235987755982988, -0.5235987755982988, 0.5235987755982988, -0.5235987755982988, 0, 0, 0, 3.141592653589793, 0, 0, 0, 0]


##LQR for planar Quadruped with Inverted Pendulum

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=096cffe7-416e-4d51-a471-5fd526ec8fab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>

In [10]:
def cartpole_balancing( x_star, Q, R):
    """
    Arguments:
        cartpole_urdf: str
            a string that contains a urdf description of the system
        x_star: array
            a fixed point of the system; the desired equilibrium point
            we want the system to reach
        Q: state cost matrix in LQR
        R: input cost matrix in LQR

    simulates and generates an animation for the lqr controlled system
    """

    builder = DiagramBuilder()

    quad, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)

    parser=Parser(quad)
    parser.package_map().Add('my_urdf','/home/ram/personal/underactuated_robotics_mit_course/programming_examples/03-Acrobots-Cart-Poles-and-Quadrotors/my_urdf')
    urdf_url = "package://my_urdf/sqwip.urdf"
    parser.AddModelsFromUrl(urdf_url)

    quad.Finalize()
    quad.set_name("quad_urdf")

    # TODO:
    # 1. obtain the context from the cartpole plant
    context=quad.CreateDefaultContext()
    # 2. set the state vector in the context to x_star
    context.get_mutable_continuous_state_vector().SetFromVector(x_star)
    # 3. fix the actuation input port to zero
    quad.get_actuation_input_port().FixValue(context, [0])
    input_i = quad.get_actuation_input_port().get_index()

    # 4. synthesize a LinearQuadraticRegulator, and add it to the builder
    lqr = LinearQuadraticRegulator(quad, context, Q, R, input_port_index=input_i)
    lqr = builder.AddSystem(lqr)
    # 5. wire cart-pole and lqr
    builder.Connect(quad.get_state_output_port(), lqr.get_input_port(0))
    builder.Connect(lqr.get_output_port(0), quad.get_actuation_input_port())

In [11]:
def simulate_and_animate(diagram, x0, sim_time=5.0, visualize=True):
    # instantiate a simulator
    simulator = Simulator(diagram)
    simulator.set_publish_every_time_step(False)  # makes sim faster
    simulator.get_mutable_integrator().set_fixed_step_mode(True)

    # start recording the video for the animation of the simulation
    visualizer = diagram.GetSubsystemByName("visualizer")
    visualizer.StartRecording(False)

    if len(x0) != diagram.GetSubsystemByName("cartpole").num_continuous_states():
        print(f"Your plant doesn't have {len(x0)} state variables.")
        return

    # reset initial time and state
    context = simulator.get_mutable_context()
    context.SetTime(0.0)
    context.SetContinuousState(x0)

    # run sim
    simulator.Initialize()
    simulator.AdvanceTo(sim_time)

    # stop video
    visualizer.PublishRecording()
    visualizer.DeleteRecording()

In [12]:
# simulate and animate the lqr controller for single-pendulum cart-pole
size=len(x_star_system)

Q = np.diag(np.full(size, 10))
R = np.eye(4)
x0 = x_star_system
single_cartpole_diagram = cartpole_balancing(
     x_star_system, Q, R
)
simulate_and_animate(
    single_cartpole_diagram,
    x0,
    sim_time=5 if running_as_notebook else 0.1,
    visualize=running_as_notebook,
)

RuntimeError: System::FixInputPortTypeCheck(): expected value of type drake::systems::BasicVector<double> with size=4 for input port 'actuation' (index 4) but the actual type was drake::systems::BasicVector<double> with size=1. (System ::quad_urdf)